In [1]:
import os, sys
import numpy as np


from lace.emulator.nn_architecture import MDNemulator_polyfit
from lace.emulator.nn_emulator import NNEmulator
from lace.emulator.gp_emulator import GPEmulator
from lace.emulator.emulator import P1D_emulator


/data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# our modules
from lace.cosmo import fit_linP
from lace.emulator import poly_p1d
from lace.archive import pnd_archive
from lace.archive import interface_archive

In [3]:
emuparams = ['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']

# CREATE TRAINING AND TESTING ARCHIVE

In [11]:
archive = pnd_archive.archivePND(sim_suite="Cabayol23")
archive.get_training_data()
len(archive.training_data)

9900

In [5]:
archive_test = pnd_archive.archivePND(sim_suite="Cabayol23", pick_sim='central')
archive_test.get_testing_data()
len(archive_test.testing_data)

11

In [6]:
kMpc = archive_test.testing_data[0]['k_Mpc'][1:43]

# TRAIN NEURAL NETWORK EMULATOR  

## The neural network emulator supposrts running both on GPU and CPU. No argument is needed, f the gpu is available, it will use it. Otherwise, the emulator trains on CPUs

## The neural network emulator uses the following default parameters:

- paramList=['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']
- nepochs=100
- step_size=75
- kmax_Mpc=4
- ndeg=5. For the extended version (kmax_Mpc=8), set to 7
- initial_weights=True. Always starts from the same iniial parameters. These were selected randomly. Set to False if random initialization is required.

## Additioanlly, we must provide either an archive 

In [7]:
emulator = NNEmulator(archive=archive, nepochs=1)

Selected custome training set
Loading emulator using a specific archive
start the training of the emulator
Training network on 9900
Emualtor trained in 0.9789836406707764 seconds


## or a training_set label

In [24]:
emulator = NNEmulator(training_set='Cabayol23', nepochs=1)

Selected pre-tuned training set
start the training of the emulator
Training network on 9900
Emualtor trained in 1.4864706993103027 seconds


## We can also load a pre-trained emulator, providing the path of the saved network parameters:

In [12]:
emulator = NNEmulator(archive=archive,nepochs=1, model_path='NNmodels/NNEmulator_LaCEHC.pt', train=False)

Selected custome training set
Loading emulator using a specific archive
Model loaded. No training needed


### Please ensure that the emulator parameters used during training are the same as the ones you are using for testing when loading a pre-trained emulator.


## Once the emulator is trained, to emulate the p1d given some model parameters:

In [15]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

# GAUSSIAN PROCESS EMULATOR

## The Gaussian process emulator uses the following default parameters:

- paramList=['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']
- kmax_Mpc=10
- ndeg=4

In [20]:
archive = pnd_archive.archivePND(sim_suite="Pedersen21")
archive.get_training_data()
len(archive.training_data)

330

## We must also provide either a training_set label

In [21]:
emulator = GPEmulator(training_set='Pedersen21')

 /data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


Training GP on 330 points
GPs optimised in 1.44 seconds


## or an archive

In [22]:
emulator = GPEmulator(archive=archive)

Training GP on 330 points
GPs optimised in 1.09 seconds


## Once the emulator is trained, to emulate the p1d given some model parameters:

In [ ]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

In [9]:
emulator = GPEmulator(training_set='Pedersen21')

 /data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


Training GP on 330 points
GPs optimised in 0.82 seconds


In [10]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)